In [ ]:
import scipy
import scipy.optimize
import numpy as np
import pickle
from matplotlib import pyplot as plt

%matplotlib widget
plt.close("all")

In [ ]:
class Solution:
    def __init__(self, T, X):
        self.T = T
        self.X = X

    def plot(self, **kwargs):
        plt.plot(self.T, self.X[:,0], 'r', **kwargs)
        plt.plot(self.T, self.X[:,1], 'b', **kwargs)

    def compare_to(self, other):
        # Find maximum differnce
        diff = self.X[:, 0] - other.X[:, 0]
        # diff_max = np.max(diff)
        # diff_min = np.min(diff)
        # return diff_max if diff_max >= -diff_min else diff_min
        return np.sum(diff)

    def compare_to_1d(self,other):
        #e.g. compare the peak values of prey
        assert other.T[0] == self.T[0] #check if both start times are equal
        assert other.T[-1] == self.T[-1] #check if both end times are equal
        return max(other.X[:,0]) - max(self.X[:,0])

    def compare_to_2d(self,other):
        #e.g. compare the peak values of prey and predator
        assert other.T[0]==self.T[0] #check if both start times are equal
        assert other.T[-1] == self.T[-1] #check if both end times are equal
        return np.array([max(other.X[:,0])-max(self.X[:,0]),max(other.X[:,1])-max(self.X[:,1])])

    def save(self,filename):
        with open(filename,'wb') as f:
            pickle.dump(self,f)

class PredPreyModel:
    def __init__(self, x0, y0, alpha, beta, gamma, delta):
        self.x0 = x0
        self.y0 = y0
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta

    def rhs(self, t, x):
        xy = x[0]*x[1]
        dx = x[0]*self.alpha-xy*self.beta
        dy = xy*self.delta-x[1]*self.gamma
        return np.array([dx,dy])

    def solve(self, t0, tend):
        maxstep = (tend-t0)/200
        X0 = np.array([self.x0,self.y0])
        rk = scipy.integrate.RK45(self.rhs, t0=t0, y0 = X0, t_bound=tend, max_step = maxstep)
        T = [t0]
        X=[X0]
        while T[-1] < tend:
            rk.step()
            X.append(rk.y)
            T.append(rk.t)
        return Solution(np.array(T),np.array(X))

def load_solution(filename):
    with open(filename,'rb') as f:
        return pickle.load(f)

In [ ]:
ppmodel = PredPreyModel(0.1, 0.2, 0.1, 0.1, 0.1, 0.1)
s1 = ppmodel.solve(0, 80)
try:
    s2 = load_solution('solution_alpha15.pickle')
except:
    print('create solution')
    ppmodel.alpha=0.15
    s2 = ppmodel.solve(0, 80)
    s2.save('solution_alpha15.pickle')

c1 = s1.compare_to_1d(s2)
print(f'prey peak of s2 is {c1:.01f} higher than s1')
c2 = s1.compare_to_2d(s2)
print(f'prey/pred peak of s2 is {c2[0]:.01f}/{c2[1]:.01f} higher than s1')
plt.figure()
s1.plot()
s2.plot(linestyle='dashed')
plt.grid()
plt.show()

# 1D Bisection for alpha parameter

In [ ]:
model = PredPreyModel(0.1, 0.2, 0.1, 0.1, 0.1, 0.1)
unknown_solution = load_solution('solution_alpha15.pickle')

# The error function.
def error_func(x):
    # Set the alpha value to the value of x, and generate a solution.
    # Then compare the result with the unknown solution.
    model.alpha = x
    solution = model.solve(0, 80)
    return solution.compare_to(unknown_solution)

# Search alpha value via bisection (1D).
bisect_min = 0.1
bisect_max = 0.173
(alpha, result) = scipy.optimize.bisect(error_func, bisect_min, bisect_max, full_output=True, rtol=1e-15)
print(f"Found value of alpha paramter: {alpha:.6f}, value: {error_func(alpha):.9f}")
print(f"Converged: {result.converged}   Function calls: {result.function_calls}   Iterations: {result.iterations}")
alpha = 0.15

# Plot the error function.
alpha_v = np.linspace(bisect_min, bisect_max, 100)
plt.figure()
plt.scatter(alpha, error_func(alpha), c="tab:orange")
plt.plot(alpha_v, np.vectorize(error_func)(alpha_v))
plt.axvline(bisect_min, color="red")
plt.axvline(bisect_max, color="red")
plt.xlabel("alpha")
plt.ylabel("error")
plt.title("Error function")
plt.grid()
plt.show()